# PDF Text Extraction with LLM Integration

This notebook demonstrates a comprehensive approach to extracting structured data from PDF documents, specifically focusing on OAB (Brazilian Bar Association) cards and system screens. We'll combine traditional text extraction methods with modern LLM integration for robust document processing.

## Objectives:
- Extract text from PDFs using multiple libraries
- Apply heuristic and regex-based extraction
- Use Named Entity Recognition (NER) 
- Integrate with OpenAI's LLM for intelligent extraction
- Build a complete extraction pipeline with validation

## 1. Install Dependencies

First, let's install all the required packages for our PDF extraction pipeline.

In [5]:
!pip install pdfplumber numpy pymupdf spacy pydantic jsonschema camelot openai python-dotenv pandas

  Using cached pymupdf-1.26.5-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ----------------- ---------------------- 1.6/3.5 MB 8.3 MB/s eta 0:00:01
     -------------------------------------- - 3.4/3.5 MB 8.7 MB/s eta 0:00:01
     ---------------------------------------- 3.5/3.5 MB 8.3 MB/s  0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached openai-2.7.1-py3-none-any.whl.metadata (29 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metada

  DEPRECATION: Building 'camelot' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'camelot'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'xlrd' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'xlrd'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'xlwt' using the legacy setup.py bdist_wheel mechanism, which will

In [41]:
!python -m spacy download pt_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 11.8 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.8 MB 11.2 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 10.3 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 10.7 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 10.9 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 10.6 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 2. Setup and Load Dataset

Let's import the necessary libraries and load our dataset configuration.

In [6]:
import json
import os
import re
import hashlib
from pathlib import Path
from typing import Dict, Any, Optional, List
import pdfplumber
import fitz  # PyMuPDF
import spacy
from pydantic import BaseModel, create_model, Field
from dotenv import load_dotenv
import openai

# Load environment variables
load_dotenv()

# Create necessary directories
os.makedirs('./files', exist_ok=True)
os.makedirs('./outputs', exist_ok=True)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [7]:
# Load dataset configuration
try:
    with open('./dataset.json', 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    print(f"📊 Dataset loaded successfully! Found {len(dataset)} entries:")
    for i, entry in enumerate(dataset):
        print(f"  {i+1}. Label: {entry['label']}, PDF: {entry['pdf_path']}")
        
except FileNotFoundError:
    print("❌ dataset.json not found in current directory")
    dataset = []

📊 Dataset loaded successfully! Found 6 entries:
  1. Label: carteira_oab, PDF: oab_1.pdf
  2. Label: carteira_oab, PDF: oab_2.pdf
  3. Label: carteira_oab, PDF: oab_3.pdf
  4. Label: tela_sistema, PDF: tela_sistema_1.pdf
  5. Label: tela_sistema, PDF: tela_sistema_2.pdf
  6. Label: tela_sistema, PDF: tela_sistema_3.pdf


In [8]:
# Check for PDF files and show metadata for the first one
pdf_path = './files/oab_1.pdf'

if os.path.exists(pdf_path):
    print(f"✅ Found PDF: {pdf_path}")
    
    # Get file metadata
    file_stats = os.stat(pdf_path)
    print(f"📄 File size: {file_stats.st_size / 1024:.2f} KB")
    
    # Use pdfplumber to get PDF metadata
    with pdfplumber.open(pdf_path) as pdf:
        print(f"📖 Number of pages: {len(pdf.pages)}")
        print(f"📏 Page size (first page): {pdf.pages[0].width} x {pdf.pages[0].height}")
        
        # Show metadata if available
        if pdf.metadata:
            print("📋 PDF Metadata:")
            for key, value in pdf.metadata.items():
                print(f"  {key}: {value}")
else:
    print(f"❌ PDF file not found: {pdf_path}")
    print("Please ensure the PDF files are in the './files/' directory")

✅ Found PDF: ./files/oab_1.pdf
📄 File size: 822.75 KB
📖 Number of pages: 1
📏 Page size (first page): 1056 x 552


## 3. Basic PDF Extraction with pdfplumber

Let's extract text and words with their coordinates from the first page using pdfplumber.

In [11]:
def extract_text_pdfplumber(pdf_path: str) -> Dict[str, Any]:
    """Extract text using pdfplumber with coordinates."""
    
    if not os.path.exists(pdf_path):
        return {"error": f"File not found: {pdf_path}"}
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            first_page = pdf.pages[0]
            
            # Extract plain text
            text = first_page.extract_text()
            
            # Extract words with coordinates
            words = first_page.extract_words()

            # Extract table data (if any)
            tables = first_page.extract_tables()
            
            return {
                "text": text,
                "words": words,
                "tables": tables,
                "page_size": (first_page.width, first_page.height)
            }
            
    except Exception as e:
        return {"error": f"Error extracting with pdfplumber: {str(e)}"}

# Demonstrate extraction
result = extract_text_pdfplumber('./files/oab_1.pdf')

if "error" not in result:
    print("📝 Extracted text (first 500 characters):")
    print(result["text"][:500] + "..." if len(result["text"]) > 500 else result["text"])
    print("\n" + "="*50 + "\n")
    
    print("📍 First 10 words with coordinates:")
    for i, word in enumerate(result["words"][:10]):
        print(f"  {i+1}. '{word['text']}' at ({word['x0']:.1f}, {word['top']:.1f}, {word['x1']:.1f}, {word['bottom']:.1f})")
else:
    print(f"❌ {result['error']}")

📝 Extracted text (first 500 characters):
JOANA D'ARC
Inscrição Seccional Subseção
101943 PR CONSELHO SECCIONAL - PARANÁ
SUPLEMENTAR
Endereço Profissional
AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista
SÃO PAULO - SP
01310300
Telefone Profissional
SITUAÇÃO REGULAR


📍 First 10 words with coordinates:
  1. 'JOANA' at (5.0, 13.0, 88.3, 38.0)
  2. 'D'ARC' at (95.3, 13.0, 170.9, 38.0)
  3. 'Inscrição' at (5.0, 120.6, 85.0, 140.6)
  4. 'Seccional' at (156.0, 120.4, 238.4, 139.4)
  5. 'Subseção' at (305.0, 120.6, 393.9, 140.6)
  6. '101943' at (8.0, 151.8, 64.7, 168.8)
  7. 'PR' at (156.0, 152.5, 178.2, 168.5)
  8. 'CONSELHO' at (305.0, 150.8, 400.4, 167.8)
  9. 'SECCIONAL' at (405.1, 150.8, 503.4, 167.8)
  10. '-' at (508.1, 150.8, 513.8, 167.8)


## 4. PDF Extraction with PyMuPDF (fitz)

Now let's compare the results using PyMuPDF, which often provides better text extraction and bounding box information.

In [34]:
def extract_text_pymupdf(pdf_path: str) -> Dict[str, Any]:
    """Extract text using PyMuPDF with bounding boxes."""
    
    if not os.path.exists(pdf_path):
        return {"error": f"File not found: {pdf_path}"}
    

    doc = fitz.open(pdf_path)
    first_page = doc[0]
    page_size = first_page.rect
    # Extract plain text
    text = first_page.get_text()
    
    # Extract text with detailed information
    text_dict = first_page.get_text("dict")
    
    # Extract words with bounding boxes
    words = []
    for block in text_dict["blocks"]:
        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    words.append({
                        "text": span["text"],
                        "bbox": span["bbox"],  # (x0, y0, x1, y1)
                        "font": span["font"],
                        "size": span["size"]
                    })
    
    doc.close()
    
    return {
        "text": text,
        "words": words,
        "page_size": (page_size.width, page_size.height)
    }


# Demonstrate extraction
fitz_result = extract_text_pymupdf('./files/oab_1.pdf')

if "error" not in fitz_result:
    print("📝 PyMuPDF extracted text (first 500 characters):")
    print(fitz_result["text"][:500] + "..." if len(fitz_result["text"]) > 500 else fitz_result["text"])
    print("\n" + "="*50 + "\n")
    
    print("📍 First 10 words with bounding boxes (PyMuPDF):")
    for i, word in enumerate(fitz_result["words"][:10]):
        if word["text"].strip():  # Skip empty text
            bbox = word["bbox"]
            print(f"  {i+1}. '{word['text']}' at ({bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f}) - Font: {word['font']}")
    
    print("\n🔍 Comparison:")
    print(f"  pdfplumber words count: {len(result.get('words', []))}")
    print(f"  PyMuPDF words count: {len([w for w in fitz_result['words'] if w['text'].strip()])}")
    print("  PyMuPDF provides additional font information and may handle complex layouts better")
    
else:
    print(f"❌ {fitz_result['error']}")

📝 PyMuPDF extracted text (first 500 characters):
JOANA D'ARC
Inscrição
Seccional
Subseção
101943
PR
CONSELHO SECCIONAL - PARANÁ
SUPLEMENTAR
Endereço Profissional
AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista
SÃO PAULO - SP
01310300
Telefone Profissional
SITUAÇÃO REGULAR



📍 First 10 words with bounding boxes (PyMuPDF):
  1. 'JOANA D'ARC' at (5.0, 6.0, 170.9, 40.3) - Font: Helvetica
  2. 'Inscrição' at (5.0, 115.0, 85.0, 142.5) - Font: Helvetica
  3. 'Seccional' at (156.0, 115.0, 238.4, 141.1) - Font: Helvetica
  4. 'Subseção' at (305.0, 115.0, 393.9, 142.5) - Font: Helvetica
  5. '101943' at (8.0, 147.0, 64.7, 170.4) - Font: Helvetica
  6. 'PR' at (156.0, 148.0, 178.2, 170.0) - Font: Helvetica
  7. 'CONSELHO SECCIONAL - PARANÁ' at (305.0, 146.0, 588.4, 169.4) - Font: Helvetica
  8. 'SUPLEMENTAR' at (6.0, 180.0, 133.5, 203.4) - Font: Helvetica
  9. 'Endereço Profissional' at (6.0, 230.0, 190.8, 256.1) - Font: Helvetica
  10. 'AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista' a

In [158]:
# Demonstrate extraction
fitz_result = extract_text_pymupdf('./files/oab_2.pdf')

if "error" not in fitz_result:
    print("📝 PyMuPDF extracted text (first 500 characters):")
    print(fitz_result["text"][:500] + "..." if len(fitz_result["text"]) > 500 else fitz_result["text"])
    print("\n" + "="*50 + "\n")
    
    print("📍 First 10 words with bounding boxes (PyMuPDF):")
    for i, word in enumerate(fitz_result["words"][:10]):
        if word["text"].strip():  # Skip empty text
            bbox = word["bbox"]
            print(f"  {i+1}. '{word['text']}' at ({bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f}) - Font: {word['font']}")
    
    print("\n🔍 Comparison:")
    print(f"  pdfplumber words count: {len(result.get('words', []))}")
    print(f"  PyMuPDF words count: {len([w for w in fitz_result['words'] if w['text'].strip()])}")
    print("  PyMuPDF provides additional font information and may handle complex layouts better")
    
else:
    print(f"❌ {fitz_result['error']}")

📝 PyMuPDF extracted text (first 500 characters):
LUIS FILIPE ARAUJO AMARAL
Inscrição
Seccional
Subseção
101943
PR
CONSELHO SECCIONAL - PARANÁ
SUPLEMENTAR
Endereço Profissional
AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista
SÃO PAULO - SP
01310300
Telefone Profissional
SITUAÇÃO REGULAR



📍 First 10 words with bounding boxes (PyMuPDF):
  1. 'LUIS FILIPE ARAUJO AMARAL' at (10.0, 6.0, 338.5, 37.6) - Font: Helvetica
  2. 'Inscrição' at (5.0, 115.0, 85.0, 142.5) - Font: Helvetica
  3. 'Seccional' at (154.0, 115.0, 236.4, 141.1) - Font: Helvetica
  4. 'Subseção' at (303.0, 115.0, 391.9, 142.5) - Font: Helvetica
  5. '101943' at (6.0, 147.0, 62.7, 170.4) - Font: Helvetica
  6. 'PR' at (156.0, 147.0, 179.6, 170.4) - Font: Helvetica
  7. 'CONSELHO SECCIONAL - PARANÁ' at (304.0, 146.0, 604.1, 170.7) - Font: Helvetica
  8. 'SUPLEMENTAR' at (6.0, 181.0, 133.5, 204.4) - Font: Helvetica
  9. 'Endereço Profissional' at (6.0, 229.0, 190.8, 255.1) - Font: Helvetica
  10. 'AVENIDA PAULISTA, Nº 2300

## 5. Heuristic Text Extraction with Regex

Let's create regex-based functions to extract specific fields from OAB documents based on common patterns.

In [35]:
def extract_nome(text: str) -> Optional[str]:
    """Extract name from text (usually after 'Nome:' label)."""
    patterns = [
        r'Nome[:\s]+([A-ZÁÊÇ\s]+?)(?:\n|Inscrição|$)',
        r'NOME[:\s]+([A-ZÁÊÇ\s]+?)(?:\n|INSCRIÇÃO|$)',
        # Look for capitalized names at the beginning
        r'^([A-ZÁÊÇ\s]{10,50}?)(?:\n|Inscrição)',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
        if match:
            return match.group(1).strip()
    return None

def extract_inscricao(text: str) -> Optional[str]:
    """Extract registration number (typically 4-6 digits)."""
    patterns = [
        r'Inscrição[:\s]*(\d{4,6})',
        r'INSCRIÇÃO[:\s]*(\d{4,6})',
        r'Nº[:\s]*(\d{4,6})',
        r'N°[:\s]*(\d{4,6})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1)
    return None

def extract_telefone(text: str) -> Optional[str]:
    """Extract Brazilian phone number."""
    patterns = [
        r'(?:Tel|Telefone|Fone)[:\s]*(\(\d{2}\)\s*\d{4,5}-?\d{4})',
        r'(\(\d{2}\)\s*\d{4,5}-?\d{4})',
        r'(\d{2}\s*\d{4,5}-?\d{4})',
        r'Tel[:\s]*(\d{2}\s*\d{8,9})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1)
    return None

def extract_seccional(text: str) -> Optional[str]:
    """Extract seccional (2-letter state code)."""
    patterns = [
        r'Seccional[:\s]*([A-Z]{2})',
        r'SECCIONAL[:\s]*([A-Z]{2})',
        r'OAB[/\s]*([A-Z]{2})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return None

def extract_situacao(text: str) -> Optional[str]:
    """Extract professional status."""
    patterns = [
        r'Situação[:\s]*((?:Regular|Irregular|Suspenso|Ativo|Inativo)[^\n]*)',
        r'SITUAÇÃO[:\s]*((?:REGULAR|IRREGULAR|SUSPENSO|ATIVO|INATIVO)[^\n]*)',
        r'(Situação\s+Regular)',
        r'(REGULAR|IRREGULAR|SUSPENSO|ATIVO|INATIVO)',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    return None

# Test our extraction functions
sample_text = result.get("text", "") if "error" not in result else ""

if sample_text:
    print("🔍 Testing regex extraction on sample text:")
    print(f"  Nome: {extract_nome(sample_text)}")
    print(f"  Inscrição: {extract_inscricao(sample_text)}")
    print(f"  Telefone: {extract_telefone(sample_text)}")
    print(f"  Seccional: {extract_seccional(sample_text)}")
    print(f"  Situação: {extract_situacao(sample_text)}")
    
    print("\n📝 Sample text for reference:")
    print(sample_text[:300] + "..." if len(sample_text) > 300 else sample_text)
else:
    print("❌ No text available for regex testing")

🔍 Testing regex extraction on sample text:
  Nome: SUPLEMENTAR
  Inscrição: 2300
  Telefone: None
  Seccional: None
  Situação: REGULAR

📝 Sample text for reference:
JOANA D'ARC
Inscrição Seccional Subseção
101943 PR CONSELHO SECCIONAL - PARANÁ
SUPLEMENTAR
Endereço Profissional
AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista
SÃO PAULO - SP
01310300
Telefone Profissional
SITUAÇÃO REGULAR


## 6. Named Entity Recognition with spaCy

Let's use spaCy's Portuguese model to identify named entities in our text, which can complement our regex-based extraction.

In [79]:
# Load Portuguese spaCy model
try:
    nlp = spacy.load("pt_core_news_sm")
    print("✅ Portuguese spaCy model loaded successfully!")
except OSError:
    print("❌ Portuguese spaCy model not found. Trying to use a basic model...")
    try:
        nlp = spacy.blank("pt")
        print("⚠️ Using blank Portuguese model (limited NER capabilities)")
    except:
        nlp = None
        print("❌ Could not load any spaCy model")

def extract_entities_spacy(text: str) -> Dict[str, List[str]]:
    """Extract named entities using spaCy."""
    
    if not nlp:
        return {"error": "spaCy model not available"}
    
    try:
        doc = nlp(text)
        
        entities = {
            "PER": [], # PERSON
            "ORG": [], # Organizations
            "LOC": [], # Locations ("Endereço Profissional")
            "MISC": []
        }
        
        for ent in doc.ents:
            label = ent.label_
            if label in entities:
                entities[label].append({
                    "text": ent.text,
                    "start": ent.start_char,
                    "end": ent.end_char,
                    "confidence": getattr(ent, 'conf', None)
                })
            else:
                entities["MISC"].append({
                    "text": ent.text,
                    "label": label,
                    "start": ent.start_char,
                    "end": ent.end_char
                })
        
        return entities
        
    except Exception as e:
        return {"error": f"Error in spaCy processing: {str(e)}"}


# Extract text from "oab_2.pdf" using PyMuPDF
fitz_result = extract_text_pymupdf('./files/oab_2.pdf')
sample_text = fitz_result.get("text", "") if "error" not in fitz_result else ""
# Test NER on our sample text
if sample_text and nlp:
    entities_result = extract_entities_spacy(sample_text)
    
    if "error" not in entities_result:
        print("🧠 Named entities found:")
        for entity_type, entities in entities_result.items():
            if entities:
                print(f"  {entity_type}:")
                for entity in entities:
                    print(f"    - '{entity['text']}' (pos: {entity['start']}-{entity['end']})")
        
        print("\n💡 When to use NER vs Regex:")
        print("  📍 NER is better for: Person names, organizations, locations, dates")
        print("  📍 Regex is better for: Specific formats (phone, ID numbers), structured patterns")
        print("  📍 Combine both: Use NER for context, regex for precise extraction")
    else:
        print(f"❌ {entities_result['error']}")
else:
    print("⚠️ Skipping NER demonstration (no text or spaCy model available)")

✅ Portuguese spaCy model loaded successfully!
🧠 Named entities found:
  ORG:
    - 'CONSELHO SECCIONAL' (pos: 65-83)
  LOC:
    - 'Endereço Profissional
AVENIDA PAULISTA' (pos: 105-143)
    - 'Pilotis' (pos: 159-166)
    - 'Bela Vista
' (pos: 168-179)
    - 'SP' (pos: 191-193)
  MISC:
    - 'LUIS FILIPE' (pos: 0-11)
    - 'ARAUJO AMARAL
Inscrição
Seccional
Subseção' (pos: 12-54)
    - 'Nº 2300' (pos: 145-152)
    - 'Telefone Profissional
SITUAÇÃO REGULAR
' (pos: 203-242)

💡 When to use NER vs Regex:
  📍 NER is better for: Person names, organizations, locations, dates
  📍 Regex is better for: Specific formats (phone, ID numbers), structured patterns
  📍 Combine both: Use NER for context, regex for precise extraction


In [77]:
# faça um exemplo de extração de labels usando spacy
text = """Consulta de Cobrança
Pesquisar por:
Tipo:
Sistema:
Contrato:
Todos
Todos
V
Buscar
CLIENTE
CPF
Operação Selecionada
Sistema
CONSIGNADO
VIr. Parc.
2.372,64
Cidade: Mozarlândia U.F .: GO CEP: 76709970
Dados para Contato
Resumo
Cobradora
Nenhuma
Telefone da
Nenhum
Cobradora
Negativado SERASA?
Não
Negativado SPC?
Não
Status Jurídico
Nenhum
Status Notificação
Nenhum
"""
nlp = spacy.load("pt_core_news_sm")
doc = nlp(text)
for ent in doc.ents:
    print("-------")
    label = ent.ents[0].label_
    print(f"{ent.text}, {ent.label_}")


-------
Consulta de Cobrança
, MISC
-------
Sistema:
Contrato:
, MISC
-------
CLIENTE
CPF
, MISC
-------
Operação Selecionada
Sistema
CONSIGNADO
VIr, MISC
-------
Parc, PER
-------
Cidade, LOC
-------
Mozarlândia, LOC
-------
GO CEP, MISC
-------
Contato
, MISC
-------
Negativado, PER
-------
SPC, PER
-------
Status Jurídico
, PER


## 7. Result Validation

In [97]:
from typing import Dict, Any

def validate_extraction_result_simple(schema: Dict[str, str], extracted: Dict[str, Any]) -> Dict[str, Any]:
    """
    Simples: garante que só existam as chaves do schema (preenche faltantes com None),
    Lista chaves extras.
    """
    errors = []
    validated = {}

    # mantém só as chaves do schema (faltantes -> None)
    for key in schema:
        val = extracted.get(key)
        if val is None:
            validated[key] = None

    # detecta chaves inesperadas e anexa (opcional)
    extras = {k: v for k, v in extracted.items() if k not in schema}
    if extras:
        errors.append(f"unexpected keys: {', '.join(extras.keys())}")
        validated.update(extras)

    return {"valid": not errors, "data": validated, "errors": errors}

# --- exemplo de uso ---
if __name__ == "__main__":
    schema = {
        "nome": "Nome do advogado",
        "inscricao": "Número de inscrição",
        "seccional": "Seccional",
        "telefone_profissional": "Telefone"
    }

    example_data = {
        "nome": "JOÃO DA SILVA SANTOS",
        "inscricao": 12345,                # inteiro -> será convertido
        "seccional": "SP",
        "telefone_profissional": None,
        "foo": "bar"                       # chave extra
    }

    res = validate_extraction_result_simple(schema, example_data)
    print(res)


{'valid': False, 'data': {'telefone_profissional': None, 'foo': 'bar'}, 'errors': ['unexpected keys: foo']}


In [110]:
elements_data = [
    {'text': "JOANA D'ARC", 'x': 100, 'y': 50},
    {'text': "Inscrição", 'x': 50, 'y': 100},
    {'text': "Seccional", 'x': 200, 'y': 100},
    {'text': "Subseção", 'x': 350, 'y': 100},
    {'text': "101943", 'x': 50, 'y': 130},
    {'text': "PR", 'x': 200, 'y': 130},
    {'text': "CONSELHO SECCIONAL - PARANÁ", 'x': 350, 'y': 130},
    {'text': "SUPLEMENTAR", 'x': 100, 'y': 180},
    {'text': "Endereço Profissional", 'x': 50, 'y': 230},
    {'text': "AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista", 'x': 50, 'y': 260},
    {'text': "SÃO PAULO - SP", 'x': 50, 'y': 290},
    {'text': "01310300", 'x': 50, 'y': 320},
    {'text': "Telefone Profissional", 'x': 50, 'y': 350},
    {'text': "SITUAÇÃO REGULAR", 'x': 100, 'y': 450}]


elements_data = [
        {'text': "Linha 3", 'x': 0, 'y': 300},
        {'text': "Linha 1", 'x': 0, 'y': 100},
        {'text': "Linha 2", 'x': 0, 'y': 200},]

elements_data.sort(key=lambda e: (e['y'], e['x']))

# Agrupar em linhas com tolerância para pequenas diferenças em y
final_lines = []
current_line = []
line_ref_y = None
y_tolerance = 5  # Tolerância em unidades de coordenada

for elem in elements_data:
    if not all(k in elem for k in ('text', 'x', 'y')):
        raise ValueError("Elemento inválido: faltando 'text', 'x' ou 'y' chave.")
    if line_ref_y is None:
        # inicia primeira linha
        current_line.append(elem)
        line_ref_y = elem['y']
    else:
        # compara com o y do primeiro elemento da linha atual
        if abs(elem['y'] - line_ref_y) <= y_tolerance:
            current_line.append(elem)
        else:
            # Finalizar linha atual e começar nova
            current_line_sorted = sorted(current_line, key=lambda elem: elem['x'])
            line_text = " ".join([e['text'] for e in current_line_sorted])
            final_lines.append(line_text)
            # Começar nova linha
            current_line = [elem]
            line_ref_y = elem['y']

# Adicionar última linha
if current_line:
    current_line_sorted = sorted(current_line, key=lambda elem: elem['x'])
    line_text = " ".join([e['text'] for e in current_line_sorted])
    final_lines.append(line_text)
print ("\n".join(final_lines))


Linha 1
Linha 2
Linha 3


In [153]:
def extract_signature(self, elements: List[Dict[str, Any]]) -> Set[str]:
    """
    Extrai a assinatura estrutural de uma lista de elementos do unstructured.
    
    Esta versão é mais criteriosa e só adiciona no máximo UMA nova assinatura
    que não esteja em self.known_labels. A estratégia é:
    - Manter adição imediata de rótulos conhecidos e títulos.
    - Coletar candidatos que passem heurísticas estritas.
    - Atribuir uma pontuação a cada candidato com base em sinais fortes de rótulo
        (dois-pontos, presença de palavras-chaves, curta extensão, ausência de
        números, categoria etc.).
    - Selecionar o melhor candidato (se atingir threshold) e adicioná-lo — no
        máximo um novo rótulo.
    
    Args:
        elements: Lista de elementos extraídos pelo unstructured
        
    Returns:
        Conjunto de rótulos que compõem a assinatura estrutural
    """
    signature: Set[str] = set()
    candidates: List[Tuple[str, float]] = []  # (label, score)
    def token_similarity(a: str, b: str) -> float:
        ta = set(a.split())
        tb = set(b.split())
        if not ta and not tb:
            return 0.0
        inter = ta.intersection(tb)
        uni = ta.union(tb)
        return len(inter) / len(uni)

    for elem in elements:
        text = elem.get('text', '').strip()
        if not text:
            continue

        normalized_text = self._normalize_text(text)

        # Regra 1: Rótulo conhecido -> adiciona sem contagem ao limite de 1 novo rótulo
        if normalized_text in self.known_labels:
            signature.add(normalized_text)
            continue

        # Evitar valores puramente numéricos ou monetários
        if re.fullmatch(r'[\d\.,\-\/R\$ %]+', text):
            continue

        # Evitar sentenças longas (rótulos são geralmente curtos)
        if len(normalized_text.split()) > 4:
            continue

        # Heurísticas adicionais para filtrar ruído
        if not (2 <= len(normalized_text) <= 50):
            continue

        # Evitar textos que contenham muitas cifras/ dígitos
        if sum(c.isdigit() for c in normalized_text) / max(1, len(normalized_text)) > 0.2:
            continue

        # Evitar se muito semelhante a um rótulo já presente
        too_similar = False
        for s in signature:
            if token_similarity(normalized_text, s) > 0.75:
                too_similar = True
                break
        if too_similar:
            continue

        # Se possui ":", adiciona sem contagem ao limite de 1 novo rótulo
        if re.search(r':\s*$', text):
            signature.add(normalized_text)
            continue

        # Score heurístico para priorizar o melhor candidato
        category = elem.get('category', '')
        score = 0.0

        # Palavra-chave do domínio (aumenta confiança)
        domain_keywords = ['nome', 'endereco', 'telefone', 'cpf', 'cnpj', 'email', 'valor', 'data', 'vencimento', 'total', 'subtotal']
        for kw in domain_keywords:
            if kw in normalized_text:
                score += 2.0

        # Preferir textos curtos (1-3 tokens)
        tokens = normalized_text.split()
        if len(tokens) == 1:
            score += 1.2
        elif len(tokens) <= 3:
            score += 0.8

        # Penalizar presença de dígitos (provável valor)
        if re.search(r'\d', normalized_text):
            score -= 2.0

        # Penalizar se contém palavras muito comuns sem sentido (stopwords simples)
        stopwords = {'de', 'do', 'da', 'dos', 'das', 'e', 'ou', 'para', 'com', 'em', 'por'}
        stopword_ratio = sum(1 for t in tokens if t in stopwords) / max(1, len(tokens))
        if stopword_ratio > 0.5:
            score -= 1.0

        # Leve bônus se categoria sinalizou algo mas não foi Title
        if category:
            score += 0.3

        # Só considerar candidatos com score razoável
        if score > 0.0:
            candidates.append((normalized_text, score))

    # Seleciona o melhor candidato (máximo 1 novo rótulo)
    if candidates:
        # Ordena por score desc e por comprimento asc para desempate
        candidates.sort(key=lambda x: (-x[1], len(x[0])))
        best_label, best_score = candidates[0]

        # Threshold conservador para aceitar um novo rótulo
        if best_score >= 1.5:
            # Evitar duplicatas contra known_labels (por segurança)
            if best_label not in self.known_labels and best_label not in signature:
                signature.add(best_label)

    return signature

In [154]:
from core.learning.struct_matcher import StructuralMatcher
# testar extract_signature
matcher = StructuralMatcher()
text = """JOANA D'ARC
Inscrição Seccional Subseção
101943 PR CONSELHO SECCIONAL - PARANÁ
SUPLEMENTAR
Endereço Profissional
AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista
SÃO PAULO - SP
01310300
Telefone Profissional
SITUAÇÃO REGULAR"""

elements = [{'text': line, 'x': 0, 'y': i*10} for i, line in enumerate(text.split('\n'))]
signature = matcher.extract_signature(elements)
print(signature)

{'situacao regular', "joana d'arc", 'suplementar', 'telefone profissional', 'endereco profissional', 'inscricao seccional subsecao', 'sao paulo - sp'}


In [155]:
elements

[{'text': "JOANA D'ARC", 'x': 0, 'y': 0},
 {'text': 'Inscrição Seccional Subseção', 'x': 0, 'y': 10},
 {'text': '101943 PR CONSELHO SECCIONAL - PARANÁ', 'x': 0, 'y': 20},
 {'text': 'SUPLEMENTAR', 'x': 0, 'y': 30},
 {'text': 'Endereço Profissional', 'x': 0, 'y': 40},
 {'text': 'AVENIDA PAULISTA, Nº 2300 andar Pilotis, Bela Vista',
  'x': 0,
  'y': 50},
 {'text': 'SÃO PAULO - SP', 'x': 0, 'y': 60},
 {'text': '01310300', 'x': 0, 'y': 70},
 {'text': 'Telefone Profissional', 'x': 0, 'y': 80},
 {'text': 'SITUAÇÃO REGULAR', 'x': 0, 'y': 90}]

In [156]:
print(extract_signature(matcher, elements))

{'endereco profissional'}


In [169]:
test_data = {
            "oab_1": {
                "expected": {
                    "nome": "JOANA D'ARC",
                    "inscricao": "101943",
                    "seccional": "PR",
                    "subsecao": "Conselho Seccional - Paraná",
                    "categoria": "Suplementar",
                    "endereco_profissional": "Avenida Paulista, Nº 2300, andar Pilotis, Bela Vista, São Paulo - SP, 01310300",
                    "situacao": "Situação Regular"
                },
                "pdf_path": "files/oab_1.pdf"
            },
            "oab_2": {
                "expected": {
                    "nome": "LUIS FILIPE ARAUJO AMARAL",
                    "inscricao": "101943",
                    "seccional": "PR",
                    "subsecao": "Conselho Seccional - Paraná",
                    "categoria": "Suplementar",
                    "endereco_profissional": None,
                    "situacao": "Situação Regular"
                },
                "pdf_path": "files/oab_2.pdf"
            },
            "oab_3": {
                "expected": {
                    "nome": "SON GOKU",
                    "inscricao": "101943",
                    "seccional": "PR",
                    "subsecao": "Conselho Seccional - Paraná",
                    "categoria": "Suplementar",
                    "endereco_profissional": None,
                    "situacao": "Situação Regular"
                },
                "pdf_path": "files/oab_3.pdf"
            }
        }
# import llm connector
import os
import sys
from core.connectors.llm_connector import LLMConnector
from core.learning.pattern_builder import PatternBuilder
from typing import List, Dict, Any
connector = LLMConnector()
pb = PatternBuilder()
def _get_elements_from_pdf(pdf_path: str) -> List[Dict[str, Any]]:
    """
    Extrai elementos estruturados de um PDF usando o LLMConnector.
    
    Args:
        pdf_path: Caminho para o arquivo PDF
        
    Returns:
        Lista de elementos com texto e coordenadas
    """
    # Usar métodos internos do LLMConnector para extrair elementos
    raw_elements = connector._parse_pdf_elements(pdf_path)
    
    # Converter para formato esperado pelo PatternBuilder
    elements = []
    for elem in raw_elements:
        if not elem.text or not elem.text.strip():
            continue
            
        # Extrair coordenadas dos metadados
        x, y = 0, 0
        if hasattr(elem, 'metadata') and elem.metadata:
            coordinates = getattr(elem.metadata, 'coordinates', None)
            if coordinates and hasattr(coordinates, 'points'):
                if coordinates.points:
                    point = coordinates.points[0]
                    if isinstance(point, (tuple, list)) and len(point) >= 2:
                        x, y = point[0], point[1]
                    elif hasattr(point, 'x') and hasattr(point, 'y'):
                        x, y = point.x, point.y
        
        elements.append({
            "text": elem.text.strip(),
            "x": x,
            "y": y
        })
    
    return elements

for test_name, test_data in test_data.items():
        
    elements = _get_elements_from_pdf(test_data["pdf_path"])
    nome = test_data["expected"]["nome"]
    
    rule_type, rule_data, confidence = pb.learn_rule_for_field("nome", nome, elements)
    print(f"Teste: {test_name}")
    print(f"Regra aprendida para 'nome': Tipo: {rule_type}, Dados: {rule_data}, Confiança: {confidence}")
    print("-" * 50)

Teste: oab_1
Regra aprendida para 'nome': Tipo: regex, Dados: {'pattern': 'texto', 'regex': '.+'}, Confiança: 0.7
--------------------------------------------------
Teste: oab_2
Regra aprendida para 'nome': Tipo: regex, Dados: {'pattern': 'texto', 'regex': '.+'}, Confiança: 0.7
--------------------------------------------------
Teste: oab_3
Regra aprendida para 'nome': Tipo: regex, Dados: {'pattern': 'texto', 'regex': '.+'}, Confiança: 0.7
--------------------------------------------------
